In [ ]:
import pandas as pd

# Mengakses file Excel dengan path baru
file_path = 'Book5.xlsx'

# Membaca file Excel
excel_data = pd.read_excel(file_path, sheet_name=None)

# Menjadikan sheet QC sebagai database
qc_data = excel_data['QC']

# Menulis ulang pada sheet OUTPUT
output_data = qc_data.copy()  # Meng-copy data dari sheet QC ke OUTPUT

# Menulis ulang header pada sheet OUTPUT
new_headers = ['UWI', 'Date', 'Perf Status', 'Perf Top (ftMD)', 'Perf Base (ftMD)', 'PERFO SAND', 'Bottom',
               'Validasi', 'Selisih Bottom', 'T_TET1', 'T_TET2', 'T_TET3', 'T_TET4', 'T_TET5', 'T_TET6',
               'T_TET7', 'T_TET8', 'T_TET9', 'T_TET10', 'T_BKA', 'T_BKB', 'T_BKC1', 'T_BKC2', 'T_BKC3', 'T_BST']

# Memeriksa panjang header baru dan kolom pada data
if len(new_headers) == len(output_data.columns):
    output_data.columns = new_headers
else:
    print("Panjang header baru tidak sesuai dengan panjang kolom pada data. Tidak dapat melanjutkan.")
    exit()

# Menyimpan perubahan pada sheet OUTPUT
excel_data['OUTPUT'] = output_data



# Filter data berurutan pada sheet OUTPUT_UNIQUE
filter_columns = ['UWI', 'Date', 'Perf Status', 'Perf Top (ftMD)', 'Perf Base (ftMD)', 'PERFO SAND', 'Bottom',
                  'Validasi', 'Selisih Bottom', 'T_TET1', 'T_TET2', 'T_TET3', 'T_TET4', 'T_TET5', 'T_TET6',
                  'T_TET7', 'T_TET8', 'T_TET9', 'T_TET10', 'T_BKA', 'T_BKB', 'T_BKC1', 'T_BKC2', 'T_BKC3', 'T_BST']

# Urutan filter
filter_order = [('Perf Status', False), ('Perf Top (ftMD)', True), ('Date', True), ('UWI', False)]

for column, ascending in filter_order:
    output_data = output_data.sort_values(
        by=column, ascending=ascending)

# Menampilkan data pada sheet TANDA
tanda_data = excel_data['TANDA']

# Iterasi untuk mengisi nilai "P" pada kolom PERFO SAND di sheet TANDA
for index, row in tanda_data.iterrows():
    uwi = row['UWI']
    date = row['Date']
    Perf_Status =row['Perf Status']
    Perf_Top = row['Perf Top (ftMD)']
    Perf_Base = row['Perf Base (ftMD)']

    # Temukan nilai PERFO SAND yang sesuai dengan UWI dan Date dari sheet OUTPUT_UNIQUE
    perfo_sand_row = output_data.loc[
        (output_data['UWI'] == uwi) & (output_data['Date'] == date), ['PERFO SAND']]

    # Iterasi untuk mengisi "P" di bawah header yang sesuai di sheet TANDA
    for column in tanda_data.columns:
        if column in perfo_sand_row['PERFO SAND'].values:
            tanda_data.loc[index, column] = 'P'

# Menyimpan perubahan pada sheet TANDA
excel_data['TANDA'] = tanda_data

# Menyimpan ke file Excel
with pd.ExcelWriter(file_path, engine='xlsxwriter') as writer:
    for sheet, data in excel_data.items():
        data.to_excel(writer, sheet_name=sheet, index=False)